In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input
from keras.layers import Dense, Dropout, Concatenate
from keras.layers import LSTM, Embedding, Bidirectional, GRU
from keras.layers import SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.initializers import Constant
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy
import pickle
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import collections
from sklearn.utils.class_weight import compute_class_weight
from gensim import models

In [ ]:
df_train = pd.read_csv("/kaggle/input/datasetfull/DataProcessed-20241215T075659Z-001/DataProcessed/train_processed.csv")
df_val = pd.read_csv("/kaggle/input/datasetfull/DataProcessed-20241215T075659Z-001/DataProcessed/dev_processed.csv")
df_test = pd.read_csv("/kaggle/input/datasetfull/DataProcessed-20241215T075659Z-001/DataProcessed/test_processed.csv")

In [ ]:
# df_val = df_val.drop(columns='Unnamed: 0')
df_train = df_train.drop(columns='Unnamed: 0')
# df_test = df_test.drop(columns='Unnamed: 0')

In [ ]:
cmt_train = df_train['processed_cmt'].astype('str').values
cmt_val = df_val['processed_cmt'].astype('str').values
cmt_test = df_test['processed_cmt'].astype('str').values

### Load label

In [ ]:
label_sentiment_train = pickle.load(open('/kaggle/input/datasetfull/Label-20241215T075702Z-001/Label/label_sentiment_train.pkl', 'rb'))
label_sentiment_val = pickle.load(open('/kaggle/input/labeling/Label/label_sentiment_val.pkl', 'rb'))
label_sentiment_test = pickle.load(open('/kaggle/input/datasetfull/Label-20241215T075702Z-001/Label/label_sentiment_test.pkl', 'rb'))


In [ ]:
label_aspect_train = pickle.load(open('/kaggle/input/datasetfull/Label-20241215T075702Z-001/Label/label_aspect_train.pkl', 'rb'))
label_aspect_val = pickle.load(open('/kaggle/input/datasetfull/Label-20241215T075702Z-001/Label/label_aspect_val.pkl', 'rb'))
label_aspect_test = pickle.load(open('/kaggle/input/datasetfull/Label-20241215T075702Z-001/Label/label_aspect_test.pkl', 'rb'))

In [ ]:
path = "/content/drive/MyDrive/ABSA"
path_model = "/content/drive/MyDrive/ABSA/"

In [ ]:
df_train = pd.read_csv(path + "/DataProcessed/train_processed.csv")
df_test = pd.read_csv(path + "/DataProcessed/test_processed.csv")
df_val  = pd.read_csv(path + "/DataProcessed/dev_processed.csv")

In [ ]:
df_train['character'] = df_train['character'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1, 'x': -1})
df_train['content'] = df_train['content'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1, 'x': -1})
df_train['scene'] = df_train['scene'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1, 'x': -1})
df_train['sound'] = df_train['sound'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1, 'x': -1})

df_test['character'] = df_test['character'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})
df_test['content'] = df_test['content'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})
df_test['scene'] = df_test['scene'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})
df_test['sound'] = df_test['sound'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})

df_val['character'] = df_val['character'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})
df_val['content'] = df_val['content'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})
df_val['scene'] = df_val['scene'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})
df_val['sound'] = df_val['sound'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1})



/tmp/ipykernel_23/3817513826.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['character'] = df_train['character'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1, 'x': -1})
/tmp/ipykernel_23/3817513826.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['content'] = df_train['content'].replace({'P': 2, 'O': 0, 'N': 1, 'X': -1, 'x': -1})
/tmp/ipykernel_23/3817513826.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call

Get label

In [ ]:
def get_label_stage_1(df):
  # input: dataframe contains label encoded 0, 1, 2, 3
  # output:
  # 1: a binary array to show whether contains that aspect
  # 2: a three dimension array to show the sentiment of that aspect

  # label 1, detect xem co
  # detect xem co aspect do trong cau hay khong.
  is_character = df.character.replace({-1:0, 2:1, 0:1})
  is_content = df.content.replace({-1:0, 2:1, 0:1})
  is_scene = df.scene.replace({-1:0, 2:1, 0:1})
  is_sound = df.sound.replace({-1:0, 2:1, 0:1})


  dict = {
          'is_character': is_character,
          'is_content': is_content,
          'is_scene': is_scene,
          'is_sound': is_sound,
         }
  df_aspect = pd.DataFrame(dict)

  # return df_aspect.to_numpy()
  return df_aspect


def get_index_label(label_stage_df, aspect):
  return [indx_ for indx_ in label_stage_df[label_stage_df['is_{}'.format(aspect)]==1].index.values if indx_ != 0]

def get_list_index_label(label_stage_df, lst_aspect):

  lst_indx_label = {}
  for aspect in lst_aspect:
    lst_indx_label[aspect] = get_index_label(label_stage_df, aspect)
  return lst_indx_label

def get_data_stage_2(lst_indx_label, df):

  data = {}
  for aspect, indx in lst_indx_label.items():
    data[aspect] = df.iloc[indx]

  return data

def get_label_stage_2(df_input):

  df_stage_2 = df_input.copy()

  # lay tung dataset cua tung khia canh truyen vao, num_classes = 3
  y_character = tf.keras.utils.to_categorical(df_stage_2['character'].character, num_classes = 3)
  y_content = tf.keras.utils.to_categorical(df_stage_2['content'].content, num_classes = 3)
  y_scene = tf.keras.utils.to_categorical(df_stage_2['scene'].scene, num_classes = 3)
  y_sound = tf.keras.utils.to_categorical(df_stage_2['sound'].sound, num_classes = 3)

  dict2 = {'character': y_character,
          'content': y_content,
          'scene': y_scene,
          'sound': y_sound,
        }

  del df_stage_2
  return dict2

def get_cmt(dict_df, lst_aspect):
  dict_cmt = {}
  for aspect in lst_aspect:
    dict_cmt[aspect] = dict_df[aspect]['processed_cmt'].values
  return dict_cmt

def get_array_label(df):
  label_stage_1_df = get_label_stage_1(df)

  lst_aspect = [col for col in df.columns.values if col!='Cmt' if col!='processed_cmt']
  lst_indx = get_list_index_label(label_stage_1_df, lst_aspect)

  df_temp = get_data_stage_2(lst_indx, df)
  cmt_df = get_cmt(df_temp, lst_aspect)

  label_stage_2 = get_label_stage_2(df_temp)
  del label_stage_1_df, lst_aspect, df_temp
  return cmt_df, label_stage_2


In [ ]:
# Chọn các cột mong muốn từ DataFrame
label_sentiment_train = df_train[['character', 'content', 'scene', 'sound']]
label_sentiment_val = df_val[['character', 'content', 'scene', 'sound']]
label_sentiment_test = df_test[['character', 'content', 'scene', 'sound']]

# # Lưu vào file .pkl
# label_sentiment_train.to_pickle(path + '/Label/label_sentiment_train.pkl')
# label_sentiment_val.to_pickle(path + '/Label/label_sentiment_val.pkl')
# label_sentiment_test.to_pickle(path + '/Label/label_sentiment_test.pkl')

print("Đã lưu các file pickle thành công.")


Đã lưu các file pickle thành công.


In [ ]:
label_stage_1 = get_label_stage_1(df_train)
lst_aspect = [col for col in df_train.columns.values if col!='Cmt' if col!='processed_cmt']
df_temp = get_data_stage_2(get_list_index_label(label_stage_1, lst_aspect), df_train)
df_2 = get_label_stage_2(df_temp)
df_2


{'character': array([[0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        ...,
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.]]),
 'content': array([[1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        ...,
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]),
 'scene': array([[1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        ...,
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.]]),
 'sound': array([[0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        ...,
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])}

In [ ]:
label_aspect_train = get_label_stage_1(df_train)
label_aspect_val = get_label_stage_1(df_val)
label_aspect_test = get_label_stage_1(df_test)
label_aspect_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3806 entries, 0 to 3805
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   is_character  3806 non-null   int64
 1   is_content    3806 non-null   int64
 2   is_scene      3806 non-null   int64
 3   is_sound      3806 non-null   int64
dtypes: int64(4)
memory usage: 119.1 KB


In [ ]:
label_aspect_test.to_pickle(path + '/Label/label_aspect_test.pkl')
label_aspect_train.to_pickle(path + '/Label/label_aspect_train.pkl')
label_aspect_val.to_pickle(path + '/Label/label_aspect_val.pkl')

In [ ]:
cmt_train, label_sentiment_train = get_array_label(df_train)
cmt_val, label_sentiment_val = get_array_label(df_val)
cmt_test, label_sentiment_test = get_array_label(df_test)



# Create word embedding matrix stage 1

In [ ]:
def get_tokenize(train, val, test):

  tok = Tokenizer(filters='')
  tok.fit_on_texts(train)
  tok.fit_on_texts(val)
  tok.fit_on_texts(test)

  tokenized_train = tok.texts_to_sequences(train)
  tokenized_val = tok.texts_to_sequences(val)
  tokenized_test = tok.texts_to_sequences(test)

  return tok, tokenized_train, tokenized_val, tokenized_test

In [ ]:
def get_padded(max_len, tokenized_train, tokenized_val, tokenized_test):

  padded_train = pad_sequences(tokenized_train, padding = 'post', maxlen = max_len)
  padded_val = pad_sequences(tokenized_val, padding = 'post', maxlen = max_len)
  padded_test = pad_sequences(tokenized_test, padding = 'post', maxlen = max_len)

  return padded_train, padded_val, padded_test

In [ ]:
def get_embedding_matrix(embedding_dim, vocab_size, tok):

  embed_matrix = np.zeros(shape=(vocab_size, embedding_dim))
  for word, i in tok.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

  return embed_matrix

In [ ]:
w2v_path = '/kaggle/input/datasetfull/word2vec-20241215T075705Z-001/word2vec/elmo_embeddings_large.txt'
w2c_model = models.KeyedVectors.load_word2vec_format(w2v_path, binary = False)

In [ ]:
# Truy cập trực tiếp các từ trong KeyedVectors
vocab = w2c_model.key_to_index  # key_to_index thay thế cho vocab trong phiên bản mới
word_vec_dict = {}

for word in vocab:
    word_vec_dict[word] = w2c_model.get_vector(word)


In [ ]:
# Đảm bảo tất cả giá trị là chuỗi
df_train['processed_cmt'] = df_train['processed_cmt'].astype(str)
df_val['processed_cmt'] = df_val['processed_cmt'].astype(str)
df_test['processed_cmt'] = df_test['processed_cmt'].astype(str)

In [ ]:
#tokenize stage 1
tokenizer_aspect, tokenized_aspect_train, tokenized_aspect_val, tokenized_aspect_test = get_tokenize(df_train['processed_cmt'].values, df_val['processed_cmt'].values, df_test['processed_cmt'].values)

max_len = 115
#padding stage 1
padded_aspect_train, padded_aspect_val, padded_aspect_test = get_padded(max_len, tokenized_aspect_train, tokenized_aspect_val, tokenized_aspect_test)

embedding_dim = 1024
#embedding matrix stage 1
vocab_aspect_size = len(tokenizer_aspect.word_index)+1
embedding_aspect_matrix = get_embedding_matrix(embedding_dim, vocab_aspect_size, tokenizer_aspect)

# Model building

In [ ]:
def bilstm_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val, epochs=70):

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_aspect_size,
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_aspect_matrix),
                  input_length=max_len,
                  trainable=True)(input)
  lstm = Bidirectional(LSTM(units = 200, activation = 'tanh'))(embed)

  # aspect
  aspect_dense2 = Dense(128, activation='relu')(lstm)
  aspect_dropout1 = Dropout(0.2)(aspect_dense2)
  aspect_dense3 = Dense(64, activation='relu')(aspect_dropout1)
  aspect_dense4 = Dense(32, activation='relu')(aspect_dense3)
  aspect_dense5 = Dense(4, activation='sigmoid')(aspect_dense4)

  aspect_model = tf.keras.Model(inputs = input, outputs = aspect_dense5)
  aspect_model.compile(optimizer=Adam(learning_rate = 0.00001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  # Fit model
  history = aspect_model.fit(x=padded_aspect_train,
    y=label_aspect_train.to_numpy(),

    validation_data=(padded_aspect_val,
      label_aspect_val.to_numpy()),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)

  return aspect_model

def bilstm_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val, epochs=70):

  # polarity

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_sentiment_size[aspect],
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_sentiment_matrix[aspect]),
                  input_length=max_len,
                  trainable=True)(input)
  lstm = Bidirectional(LSTM(units = 200, activation = 'tanh'))(embed)

  sentiment_dense2 = Dense(128, activation='relu')(lstm)
  sentiment_dropout1 = Dropout(0.2)(sentiment_dense2)
  sentiment_dense3 = Dense(64, activation='relu')(sentiment_dropout1)
  sentiment_dense4 = Dense(32, activation='relu')(sentiment_dense3)
  out_sentiment = Dense(units = 3, activation = 'softmax')(sentiment_dense4)

  sentiment_model[aspect] = tf.keras.Model(inputs = input, outputs = out_sentiment)
  sentiment_model[aspect].compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model

  history = sentiment_model[aspect].fit(x=padded_sentiment_train[aspect],
    y=label_sentiment_train[aspect],

    validation_data=(padded_sentiment_val[aspect],
      label_sentiment_val[aspect]),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)


  return sentiment_model[aspect]

In [ ]:
label_aspect_train = label_aspect_train.astype('float')

In [ ]:
max_len=padded_aspect_train.shape[1]
aspect_model = bilstm_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val)

Epoch 1/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - acc: 0.1988 - loss: 0.6860 - val_acc: 0.2080 - val_loss: 0.6791
Epoch 2/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.2022 - loss: 0.6757 - val_acc: 0.2122 - val_loss: 0.6698
Epoch 3/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.2042 - loss: 0.6669 - val_acc: 0.2080 - val_loss: 0.6611
Epoch 4/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.2027 - loss: 0.6531 - val_acc: 0.2101 - val_loss: 0.6528
Epoch 5/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.1996 - loss: 0.6440 - val_acc: 0.2101 - val_loss: 0.6460
Epoch 6/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.2039 - loss: 0.6407 - val_acc: 0.2122 - val_loss: 0.6403
Epoch 7/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.2023 - loss: 0.6307 - val_acc: 0.2185 - val_loss: 0.6350
Epoch 8/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - acc: 0.2017 - loss: 0.6269 - val_acc: 0.2332 - val_loss: 0.6297
Epoch 9/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - acc: 0.2073

In [ ]:
max_len=padded_aspect_train.shape[1]
aspect_model = bilstm_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 103ms/step - acc: 0.1929 - loss: 0.6953 - val_acc: 0.2290 - val_loss: 0.6877
Epoch 2/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - acc: 0.2048 - loss: 0.6838 - val_acc: 0.2374 - val_loss: 0.6780
Epoch 3/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - acc: 0.2211 - loss: 0.6743 - val_acc: 0.2542 - val_loss: 0.6677
Epoch 4/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - acc: 0.2224 - loss: 0.6633 - val_acc: 0.2647 - val_loss: 0.6560
Epoch 5/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - acc: 0.2290 - loss: 0.6504 - val_acc: 0.2773 - val_loss: 0.6437
Epoch 6/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - acc: 0.2440 - loss: 0.6334 - val_acc: 0.2815 - val_loss: 0.6306
Epoch 7/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - acc: 0.2505 - loss: 0.6203 - val_acc: 0.3067 - val_loss: 0.6184
Epoch 8/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - acc: 0.2653 - loss: 0.6141 - val_acc: 0.3172 - val_loss: 0.6080
Epoch 9/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - acc: 0.296

In [ ]:
save_path = f'{path}/BiLSTM/stl_BiLSTM_aspect.h5'
aspect_model.save(save_path)

In [ ]:
# predict
pred_train = aspect_model.predict(padded_aspect_train)
pred_val = aspect_model.predict(padded_aspect_val)
pred_test = aspect_model.predict(padded_aspect_test)

df_train_pred = round(pd.DataFrame(pred_train), 0)
df_train_true_ = label_aspect_train
df_train_pred.columns = df_train_true_.columns

df_val_pred = round(pd.DataFrame(pred_val), 0)
df_val_true_ = label_aspect_val
df_val_pred.columns = df_val_true_.columns

df_test_pred = round(pd.DataFrame(pred_test), 0)
df_test_true_ = label_aspect_test
df_test_pred.columns = df_test_true_.columns

120/120 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [ ]:
def get_pd_report(label_aspect_train, df_test_true_, df_test_pred):

  aspect_result_bilstm_report = {}
  for col in label_aspect_train.columns:
    aspect_result_bilstm = classification_report(df_test_true_[col], df_test_pred[col], output_dict=True)
    aspect_result_bilstm_report[col] = pd.DataFrame(aspect_result_bilstm).T
    aspect_result_bilstm_report[col]['aspect'] = col

  output_aspect_report = pd.DataFrame()
  for indx, val in aspect_result_bilstm_report.items():
    output_aspect_report = pd.concat([output_aspect_report, aspect_result_bilstm_report[indx]])

  return output_aspect_report

In [ ]:
aspect_result_bilstm_report = {}
for col in label_aspect_train.columns:
  print(col)
  aspect_result_bilstm = classification_report(df_test_true_[col], df_test_pred[col], output_dict=True)
  print(aspect_result_bilstm)

  aspect_result_bilstm_report[col] = pd.DataFrame(aspect_result_bilstm).T
  aspect_result_bilstm_report[col]['aspect'] = col

output_aspect_report = pd.DataFrame()
for indx, val in aspect_result_bilstm_report.items():
  output_aspect_report = pd.concat([output_aspect_report, aspect_result_bilstm_report[indx]])

is_character
{'0': {'precision': 0.8493975903614458, 'recall': 0.9215686274509803, 'f1-score': 0.8840125391849529, 'support': 306.0}, '1': {'precision': 0.8344827586206897, 'recall': 0.7076023391812866, 'f1-score': 0.7658227848101266, 'support': 171.0}, 'accuracy': 0.8448637316561844, 'macro avg': {'precision': 0.8419401744910677, 'recall': 0.8145854833161335, 'f1-score': 0.8249176619975398, 'support': 477.0}, 'weighted avg': {'precision': 0.8440507638883445, 'recall': 0.8448637316561844, 'f1-score': 0.8416426272392604, 'support': 477.0}}
is_content
{'0': {'precision': 0.8211678832116789, 'recall': 0.8241758241758241, 'f1-score': 0.8226691042047533, 'support': 273.0}, '1': {'precision': 0.7635467980295566, 'recall': 0.7598039215686274, 'f1-score': 0.7616707616707616, 'support': 204.0}, 'accuracy': 0.7966457023060797, 'macro avg': {'precision': 0.7923573406206177, 'recall': 0.7919898728722258, 'f1-score': 0.7921699329377574, 'support': 477.0}, 'weighted avg': {'precision': 0.79652490338

In [ ]:
output_aspect_report

,precision,recall,f1-score,support,aspect
0,0.849398,0.921569,0.884013,306.000000,is_character
1,0.834483,0.707602,0.765823,171.000000,is_character
accuracy,0.844864,0.844864,0.844864,0.844864,is_character
macro avg,0.841940,0.814585,0.824918,477.000000,is_character
weighted avg,0.844051,0.844864,0.841643,477.000000,is_character
0,0.821168,0.824176,0.822669,273.000000,is_content
1,0.763547,0.759804,0.761671,204.000000,is_content
accuracy,0.796646,0.796646,0.796646,0.796646,is_content
macro avg,0.792357,0.791990,0.792170,477.000000,is_content
weighted avg,0.796525,0.796646,0.796582,477.000000,is_content


In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
plt.plot(history.history['out_character_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_out_character_accuracy'], label='Validation Accuracy')
plt.legend()

- Stage 2 : Polarity

In [ ]:
lst_aspect = []
for col in label_aspect_train.columns.values:
  if col!='is_others':
    lst_aspect.append(col.split('_')[1])

In [ ]:
#tokenize stage 2
tokenized_sentiment_train = {}
tokenized_sentiment_val = {}
tokenized_sentiment_test = {}
tokenizer_sentiment = {}

for aspect in lst_aspect:
  tokenizer_sentiment[aspect], tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect] = get_tokenize(cmt_train[aspect], cmt_val[aspect], cmt_test[aspect])

#padding stage 2
padded_sentiment_train = {}
padded_sentiment_val = {}
padded_sentiment_test = {}
for aspect in lst_aspect:
  padded_sentiment_train[aspect], padded_sentiment_val[aspect], padded_sentiment_test[aspect] = get_padded(max_len, tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect])

#embedding matrix stage 2
vocab_sentiment_size = {}
embedding_sentiment_matrix = {}
for aspect in lst_aspect:
  vocab_sentiment_size[aspect]  = len(tokenizer_sentiment[aspect].word_index)+1
  embedding_sentiment_matrix[aspect] = get_embedding_matrix(embedding_dim, vocab_sentiment_size[aspect], tokenizer_sentiment[aspect])

In [ ]:
# training model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = bilstm_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val)
  sentiment_model[aspect].save('/kaggle/working/BiLSTM/stl_BiLSTM_polarity_{aspect}.h5')

Epoch 1/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 114ms/step - acc: 0.3396 - loss: 0.6714 - val_acc: 0.5655 - val_loss: 0.6267
Epoch 2/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.5549 - loss: 0.6148 - val_acc: 0.6667 - val_loss: 0.5666
Epoch 3/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.6295 - loss: 0.5609 - val_acc: 0.6786 - val_loss: 0.5270
Epoch 4/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.6490 - loss: 0.5283 - val_acc: 0.6786 - val_loss: 0.5005
Epoch 5/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.6540 - loss: 0.5045 - val_acc: 0.6667 - val_loss: 0.4875
Epoch 6/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.6461 - loss: 0.4986 - val_acc: 0.6667 - val_loss: 0.4793
Epoch 7/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.6509 - loss: 0.4807 - val_acc: 0.6786 - val_loss: 0.4703
Epoch 8/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.6785 - loss: 0.4545 - val_acc: 0.6786 - val_loss: 0.4634
Epoch 9/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - acc: 0.680

In [ ]:
# load model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = keras.models.load_model('/kaggle/working/BiLSTM/stl_BiLSTM_polarity_{aspect}.h5')

In [ ]:
# predict
df_sentiment_train_pred = {}
df_sentiment_train_true_ = {}

df_sentiment_val_true_ = {}
df_sentiment_val_pred = {}

df_sentiment_test_true_ = {}
df_sentiment_test_pred = {}


# for aspect in lst_aspect:
for aspect in lst_aspect:
  pred_sentiment_train = sentiment_model[aspect].predict(padded_sentiment_train[aspect])
  pred_sentiment_val = sentiment_model[aspect].predict(padded_sentiment_val[aspect])
  pred_sentiment_test = sentiment_model[aspect].predict(padded_sentiment_test[aspect])

  df_sentiment_train_pred[aspect] = np.argmax(pred_sentiment_train, axis=1)
  df_sentiment_train_true_[aspect] = np.argmax(label_sentiment_train[aspect], axis = 1)

  df_sentiment_val_pred[aspect] = np.argmax(pred_sentiment_val, axis=1)
  df_sentiment_val_true_[aspect] = np.argmax(label_sentiment_val[aspect], axis = 1)

  df_sentiment_test_pred[aspect] = np.argmax(pred_sentiment_test, axis=1)
  df_sentiment_test_true_[aspect] = np.argmax(label_sentiment_test[aspect], axis=1)

42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [ ]:
def get_pd_report_sentiment(df_sentiment_test_true_, df_sentiment_test_pred):

  sentiment_result_bilstm = {}
  # for aspect in lst_aspect:
  for aspect in lst_aspect:
      sentiment_result_bilstm[aspect] = pd.DataFrame(classification_report(df_sentiment_test_true_[aspect].tolist(), df_sentiment_test_pred[aspect].tolist(), output_dict=True)).T
      sentiment_result_bilstm[aspect]['aspect'] = aspect

  output_sentiment_report = pd.DataFrame()
  for indx, val in sentiment_result_bilstm.items():
    output_sentiment_report = pd.concat([output_sentiment_report, sentiment_result_bilstm[indx]])

  return output_aspect_report

In [ ]:
sentiment_result_bilstm = {}
# for aspect in lst_aspect:
for aspect in lst_aspect:
    sentiment_result_bilstm[aspect] = pd.DataFrame(classification_report(df_sentiment_test_true_[aspect].tolist(), df_sentiment_test_pred[aspect].tolist(), output_dict=True)).T
    sentiment_result_bilstm[aspect]['aspect'] = aspect

output_sentiment_report = pd.DataFrame()
for indx, val in sentiment_result_bilstm.items():
  output_sentiment_report = pd.concat([output_sentiment_report, sentiment_result_bilstm[indx]])

In [ ]:
output_sentiment_report

,precision,recall,f1-score,support,aspect
0,0.583333,0.129630,0.212121,54.000000,character
1,0.086207,0.227273,0.125000,22.000000,character
2,0.590000,0.627660,0.608247,94.000000,character
accuracy,0.417647,0.417647,0.417647,0.417647,character
macro avg,0.419847,0.328187,0.315123,170.000000,character
weighted avg,0.522686,0.417647,0.419881,170.000000,character
0,0.200000,0.075000,0.109091,40.000000,content
1,0.324324,0.400000,0.358209,60.000000,content
2,0.582609,0.644231,0.611872,104.000000,content
accuracy,0.460784,0.460784,0.460784,0.460784,content


### BiGRU

In [ ]:
def bigru_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val, epochs=70):

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_aspect_size,
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_aspect_matrix),
                  input_length=max_len,
                  trainable=True)(input)
  lstm = Bidirectional(GRU(units = 200, activation = 'tanh'))(embed)

  # aspect
  aspect_dense2 = Dense(128, activation='relu')(lstm)
  aspect_dropout1 = Dropout(0.2)(aspect_dense2)
  aspect_dense3 = Dense(64, activation='relu')(aspect_dropout1)
  aspect_dense4 = Dense(32, activation='relu')(aspect_dense3)
  aspect_dense5 = Dense(4, activation='sigmoid')(aspect_dense4)

  aspect_model = tf.keras.Model(inputs = input, outputs = aspect_dense5)
  aspect_model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model
  history = aspect_model.fit(x=padded_aspect_train,
    y=label_aspect_train.to_numpy(),

    validation_data=(padded_aspect_val,
      label_aspect_val.to_numpy()),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)

  return aspect_model

def bigru_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val, epochs=70):

  # polarity

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_sentiment_size[aspect],
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_sentiment_matrix[aspect]),
                  input_length=max_len,
                  trainable=True)(input)
  lstm = Bidirectional(GRU(units = 200, activation = 'tanh'))(embed)

  sentiment_dense2 = Dense(128, activation='relu')(lstm)
  sentiment_dropout1 = Dropout(0.2)(sentiment_dense2)
  sentiment_dense3 = Dense(64, activation='relu')(sentiment_dropout1)
  sentiment_dense4 = Dense(32, activation='relu')(sentiment_dense3)
  out_sentiment = Dense(units = 3, activation = 'softmax')(sentiment_dense4)

  sentiment_model[aspect] = tf.keras.Model(inputs = input, outputs = out_sentiment)
  sentiment_model[aspect].compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model

  history = sentiment_model[aspect].fit(x=padded_sentiment_train[aspect],
    y=label_sentiment_train[aspect],

    validation_data=(padded_sentiment_val[aspect],
      label_sentiment_val[aspect]),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)


  return sentiment_model[aspect]

In [ ]:
max_len=padded_aspect_train.shape[1]
aspect_model = bigru_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val)

save_path = f'/kaggle/working/BiGRU/stl_BiGRU_aspect.h5'
aspect_model.save(save_path)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - acc: 0.2492 - loss: 0.6614 - val_acc: 0.4055 - val_loss: 0.6218
Epoch 2/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - acc: 0.4194 - loss: 0.6004 - val_acc: 0.4454 - val_loss: 0.5789
Epoch 3/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - acc: 0.4397 - loss: 0.5596 - val_acc: 0.4517 - val_loss: 0.5507
Epoch 4/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - acc: 0.4692 - loss: 0.5264 - val_acc: 0.4685 - val_loss: 0.5296
Epoch 5/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - acc: 0.4854 - loss: 0.5052 - val_acc: 0.4958 - val_loss: 0.5187
Epoch 6/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - acc: 0.5044 - loss: 0.4922 - val_acc: 0.4979 - val_loss: 0.5048
Epoch 7/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - acc: 0.5135 - loss: 0.4716 - val_acc: 0.4580 - val_loss: 0.4934
Epoch 8/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - acc: 0.5051 - loss: 0.4542 - val_acc: 0.5000 - val_loss: 0.4803
Epoch 9/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - acc: 0.5375

In [ ]:
# load model
save_path = '/kaggle/working/BiGRU/stl_BiGRU_aspect.h5'
aspect_model = keras.models.load_model(save_path)

In [ ]:
# predict
pred_train = aspect_model.predict(padded_aspect_train)
pred_val = aspect_model.predict(padded_aspect_val)
pred_test = aspect_model.predict(padded_aspect_test)

df_train_pred = round(pd.DataFrame(pred_train), 0)
df_train_true_ = label_aspect_train
df_train_pred.columns = df_train_true_.columns

df_val_pred = round(pd.DataFrame(pred_val), 0)
df_val_true_ = label_aspect_val
df_val_pred.columns = df_val_true_.columns

df_test_pred = round(pd.DataFrame(pred_test), 0)
df_test_true_ = label_aspect_test
df_test_pred.columns = df_test_true_.columns

119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [ ]:
for col in label_aspect_train.columns:
  print(col)
  aspect_result_bigru = classification_report(df_test_true_[col], df_test_pred[col])
  print(aspect_result_bigru)

is_character
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       306
           1       0.80      0.79      0.79       171

    accuracy                           0.85       477
   macro avg       0.84      0.84      0.84       477
weighted avg       0.85      0.85      0.85       477

is_content
              precision    recall  f1-score   support

           0       0.86      0.86      0.86       273
           1       0.82      0.81      0.81       204

    accuracy                           0.84       477
   macro avg       0.84      0.84      0.84       477
weighted avg       0.84      0.84      0.84       477

is_scene
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       309
           1       0.70      0.70      0.70       168

    accuracy                           0.79       477
   macro avg       0.77      0.77      0.77       477
weighted avg       0.79      0.79      0.

In [ ]:
output_aspect_report = get_pd_report(label_aspect_train, df_test_true_, df_test_pred)
output_aspect_report

,precision,recall,f1-score,support,aspect
0,0.883117,0.888889,0.885993,306.000000,is_character
1,0.798817,0.789474,0.794118,171.000000,is_character
accuracy,0.853249,0.853249,0.853249,0.853249,is_character
macro avg,0.840967,0.839181,0.840056,477.000000,is_character
weighted avg,0.852896,0.853249,0.853057,477.000000,is_character
0,0.858182,0.864469,0.861314,273.000000,is_content
1,0.816832,0.808824,0.812808,204.000000,is_content
accuracy,0.840671,0.840671,0.840671,0.840671,is_content
macro avg,0.837507,0.836646,0.837061,477.000000,is_content
weighted avg,0.840497,0.840671,0.840569,477.000000,is_content


- Stage 2 : Polarity

In [ ]:
lst_aspect = []
for col in label_aspect_train.columns.values:
  if col!='is_others':
    lst_aspect.append(col.split('_')[1])

In [ ]:
#tokenize stage 2
tokenized_sentiment_train = {}
tokenized_sentiment_val = {}
tokenized_sentiment_test = {}
tokenizer_sentiment = {}

for aspect in lst_aspect:
  tokenizer_sentiment[aspect], tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect] = get_tokenize(cmt_train[aspect], cmt_val[aspect], cmt_test[aspect])

#padding stage 2
padded_sentiment_train = {}
padded_sentiment_val = {}
padded_sentiment_test = {}
for aspect in lst_aspect:
  padded_sentiment_train[aspect], padded_sentiment_val[aspect], padded_sentiment_test[aspect] = get_padded(max_len, tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect])

#embedding matrix stage 2
vocab_sentiment_size = {}
embedding_sentiment_matrix = {}
for aspect in lst_aspect:
  vocab_sentiment_size[aspect]  = len(tokenizer_sentiment[aspect].word_index)+1
  embedding_sentiment_matrix[aspect] = get_embedding_matrix(embedding_dim, vocab_sentiment_size[aspect], tokenizer_sentiment[aspect])

In [ ]:
# training model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = bigru_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val)
  sentiment_model[aspect].save(f'/kaggle/working/BiGRU/stl_BiGRU_polarity_{aspect}.h5')

Epoch 1/70


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - acc: 0.3195 - loss: 0.6813 - val_acc: 0.2798 - val_loss: 0.6326
Epoch 2/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.3227 - loss: 0.6225 - val_acc: 0.3274 - val_loss: 0.6006
Epoch 3/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.3868 - loss: 0.5885 - val_acc: 0.5000 - val_loss: 0.5739
Epoch 4/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.5226 - loss: 0.5654 - val_acc: 0.6071 - val_loss: 0.5492
Epoch 5/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - acc: 0.6029 - loss: 0.5458 - val_acc: 0.6369 - val_loss: 0.5236
Epoch 6/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.6512 - loss: 0.5226 - val_acc: 0.6607 - val_loss: 0.5031
Epoch 7/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.6497 - loss: 0.4951 - val_acc: 0.6667 - val_loss: 0.4884
Epoch 8/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.6550 - loss: 0.4938 - val_acc: 0.6607 - val_loss: 0.4808
Epoch 9/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - acc: 0.6615 - loss: 0.

In [ ]:
# predict
df_sentiment_train_pred = {}
df_sentiment_train_true_ = {}

df_sentiment_val_true_ = {}
df_sentiment_val_pred = {}

df_sentiment_test_true_ = {}
df_sentiment_test_pred = {}


# for aspect in lst_aspect:
for aspect in lst_aspect:
  pred_sentiment_train = sentiment_model[aspect].predict(padded_sentiment_train[aspect])
  pred_sentiment_val = sentiment_model[aspect].predict(padded_sentiment_val[aspect])
  pred_sentiment_test = sentiment_model[aspect].predict(padded_sentiment_test[aspect])

  df_sentiment_train_pred[aspect] = np.argmax(pred_sentiment_train, axis=1)
  df_sentiment_train_true_[aspect] = np.argmax(label_sentiment_train[aspect], axis = 1)

  df_sentiment_val_pred[aspect] = np.argmax(pred_sentiment_val, axis=1)
  df_sentiment_val_true_[aspect] = np.argmax(label_sentiment_val[aspect], axis = 1)

  df_sentiment_test_pred[aspect] = np.argmax(pred_sentiment_test, axis=1)
  df_sentiment_test_true_[aspect] = np.argmax(label_sentiment_test[aspect], axis=1)

sentiment_result_bigru = {}
# for aspect in lst_aspect:
for aspect in lst_aspect:
    sentiment_result_bigru[aspect] = classification_report(df_sentiment_test_true_[aspect].tolist(), df_sentiment_test_pred[aspect].tolist())
    print(sentiment_result_bigru[aspect])

42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
              precision    recall  f1-score   support

           0       0.80      0.65      0.71        54
           1       0.64      0.41      0.50        22
           2       0.75      0.89      0.82        94

    accuracy                           0.75       170
   macro avg       0.73      0.65      0.68       170
weighted avg       0.75      0.75      0.74       170

              precision    recall  f1-score   support

           0       0.36      0.12      0.19        40
           1       0.59      0.58      0.59   

### BiGRU+CONV

In [ ]:
def bigru_conv_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val, epochs=70):

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_aspect_size,
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_aspect_matrix),
                  input_length=max_len,
                  trainable=True)(input)

  dropout1 = SpatialDropout1D(0.2)(embed)

  lstm = Bidirectional(GRU(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

  avg_pool1 = GlobalAveragePooling1D()(conv)
  max_pool1 = GlobalMaxPooling1D()(conv)


  concat = Concatenate(axis=-1)([avg_pool1, max_pool1])

  # aspect
  aspect_dense2 = Dense(128, activation='relu')(concat)
  aspect_dropout1 = Dropout(0.2)(aspect_dense2)
  aspect_dense3 = Dense(64, activation='relu')(aspect_dropout1)
  aspect_dense4 = Dense(32, activation='relu')(aspect_dense3)
  aspect_dense5 = Dense(8, activation='sigmoid')(aspect_dense4)

  aspect_model = tf.keras.Model(inputs = input, outputs = aspect_dense5)
  aspect_model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model
  history = aspect_model.fit(x=padded_aspect_train,
    y=label_aspect_train.to_numpy(),

    validation_data=(padded_aspect_val,
      label_aspect_val.to_numpy()),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)

  return aspect_model

def bigru_conv_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val, epochs=70):

  # polarity

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_sentiment_size[aspect],
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_sentiment_matrix[aspect]),
                  input_length=max_len,
                  trainable=True)(input)

  dropout1 = SpatialDropout1D(0.2)(embed)

  lstm = Bidirectional(GRU(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

  avg_pool1 = GlobalAveragePooling1D()(conv)
  max_pool1 = GlobalMaxPooling1D()(conv)


  concat = Concatenate(axis=-1)([avg_pool1, max_pool1])

  sentiment_dense2 = Dense(128, activation='relu')(concat)
  sentiment_dropout1 = Dropout(0.2)(sentiment_dense2)
  sentiment_dense3 = Dense(64, activation='relu')(sentiment_dropout1)
  sentiment_dense4 = Dense(32, activation='relu')(sentiment_dense3)
  out_sentiment = Dense(units = 3, activation = 'softmax')(sentiment_dense4)

  sentiment_model[aspect] = tf.keras.Model(inputs = input, outputs = out_sentiment)
  sentiment_model[aspect].compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model

  history = sentiment_model[aspect].fit(x=padded_sentiment_train[aspect],
    y=label_sentiment_train[aspect],

    validation_data=(padded_sentiment_val[aspect],
      label_sentiment_val[aspect]),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)


  return sentiment_model[aspect]

In [ ]:
label_aspect_val = label_aspect_val.astype('int32')


In [ ]:
max_len=padded_aspect_train.shape[1]
aspect_model_bigru = bigru_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val)

save_path = '/kaggle/working/BiGRU_CONV/stl_BiGRU_aspect.h5'
aspect_model_bigru.save(save_path)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - acc: 0.1484 - loss: 0.6821 - val_acc: 0.2374 - val_loss: 0.6372
Epoch 2/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - acc: 0.2607 - loss: 0.6190 - val_acc: 0.3803 - val_loss: 0.5837
Epoch 3/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - acc: 0.3908 - loss: 0.5696 - val_acc: 0.4538 - val_loss: 0.5501
Epoch 4/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - acc: 0.4805 - loss: 0.5385 - val_acc: 0.4664 - val_loss: 0.5312
Epoch 5/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - acc: 0.4881 - loss: 0.5149 - val_acc: 0.4706 - val_loss: 0.5170
Epoch 6/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - acc: 0.5071 - loss: 0.4963 - val_acc: 0.4769 - val_loss: 0.5112
Epoch 7/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - acc: 0.5098 - loss: 0.4870 - val_acc: 0.5063 - val_loss: 0.5027
Epoch 8/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - acc: 0.5207 - loss: 0.4766 - val_acc: 0.4496 - val_loss: 0.4917
Epoch 9/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - acc: 0.5091

In [ ]:
# load model
save_path = '/kaggle/working/BiGRU_CONV/stl_BiGRU_aspect.h5'
aspect_model = keras.models.load_model(save_path)

In [ ]:
# predict
pred_train = aspect_model_bigru.predict(padded_aspect_train)
pred_val = aspect_model_bigru.predict(padded_aspect_val)
pred_test = aspect_model_bigru.predict(padded_aspect_test)

df_train_pred = round(pd.DataFrame(pred_train), 0)
df_train_true_ = label_aspect_train
df_train_pred.columns = df_train_true_.columns

df_val_pred = round(pd.DataFrame(pred_val), 0)
df_val_true_ = label_aspect_val
df_val_pred.columns = df_val_true_.columns

df_test_pred = round(pd.DataFrame(pred_test), 0)
df_test_true_ = label_aspect_test
df_test_pred.columns = df_test_true_.columns

for col in label_aspect_train.columns:
  print(col)
  aspect_result_bigru_conv = classification_report(df_test_true_[col], df_test_pred[col])
  print(aspect_result_bigru_conv)

119/119 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
is_character
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       306
           1       0.87      0.77      0.81       171

    accuracy                           0.87       477
   macro avg       0.87      0.85      0.86       477
weighted avg       0.87      0.87      0.87       477

is_content
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       273
           1       0.78      0.78      0.78       204

    accuracy                           0.81       477
   macro avg       0.81      0.81      0.81       477
weighted avg       0.81      0.81      0.81       477

is_scene
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       309
           1       0.70      0.70      0.70       168

    accuracy                  

In [ ]:
output_aspect_report = get_pd_report(label_aspect_train, df_test_true_, df_test_pred)
output_aspect_report

,precision,recall,f1-score,support,aspect
0,0.877301,0.934641,0.905063,306.000000,is_character
1,0.867550,0.766082,0.813665,171.000000,is_character
accuracy,0.874214,0.874214,0.874214,0.874214,is_character
macro avg,0.872425,0.850361,0.859364,477.000000,is_character
weighted avg,0.873805,0.874214,0.872298,477.000000,is_character
0,0.838235,0.835165,0.836697,273.000000,is_content
1,0.780488,0.784314,0.782396,204.000000,is_content
accuracy,0.813417,0.813417,0.813417,0.813417,is_content
macro avg,0.809362,0.809739,0.809547,477.000000,is_content
weighted avg,0.813538,0.813417,0.813474,477.000000,is_content


- Stage 2 : Polarity

In [ ]:
lst_aspect = []
for col in label_aspect_train.columns.values:
    lst_aspect.append(col.split('_')[1])

In [ ]:
#tokenize stage 2
tokenized_sentiment_train = {}
tokenized_sentiment_val = {}
tokenized_sentiment_test = {}
tokenizer_sentiment = {}

for aspect in lst_aspect:
  tokenizer_sentiment[aspect], tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect] = get_tokenize(cmt_train[aspect], cmt_val[aspect], cmt_test[aspect])

#padding stage 2
padded_sentiment_train = {}
padded_sentiment_val = {}
padded_sentiment_test = {}
for aspect in lst_aspect:
  padded_sentiment_train[aspect], padded_sentiment_val[aspect], padded_sentiment_test[aspect] = get_padded(max_len, tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect])

#embedding matrix stage 2
vocab_sentiment_size = {}
embedding_sentiment_matrix = {}
for aspect in lst_aspect:
  vocab_sentiment_size[aspect]  = len(tokenizer_sentiment[aspect].word_index)+1
  embedding_sentiment_matrix[aspect] = get_embedding_matrix(embedding_dim, vocab_sentiment_size[aspect], tokenizer_sentiment[aspect])

In [ ]:
# training model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = bigru_conv_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val)
  sentiment_model[aspect].save(f'/kaggle/working/BiGRU_CONV/stl_BiGRU_CONV_polarity_{aspect}.h5')

Epoch 1/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 104ms/step - acc: 0.3234 - loss: 0.7818 - val_acc: 0.2798 - val_loss: 0.6612
Epoch 2/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - acc: 0.3758 - loss: 0.6540 - val_acc: 0.5952 - val_loss: 0.6116
Epoch 3/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - acc: 0.5200 - loss: 0.6095 - val_acc: 0.5774 - val_loss: 0.5610
Epoch 4/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - acc: 0.5721 - loss: 0.5649 - val_acc: 0.6012 - val_loss: 0.5376
Epoch 5/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - acc: 0.5981 - loss: 0.5541 - val_acc: 0.6905 - val_loss: 0.5195
Epoch 6/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - acc: 0.6319 - loss: 0.5309 - val_acc: 0.6845 - val_loss: 0.5029
Epoch 7/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - acc: 0.6432 - loss: 0.5195 - val_acc: 0.6726 - val_loss: 0.4881
Epoch 8/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - acc: 0.6619 - loss: 0.4887 - val_acc: 0.6786 - val_loss: 0.4750
Epoch 9/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - acc: 0.657

In [ ]:
# load model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = keras.models.load_model(f'{path_model}/BiGRU_CONV/stl_BiGRU_CONV_polarity_{aspect}.h5')

In [ ]:
# predict
df_sentiment_train_pred = {}
df_sentiment_train_true_ = {}

df_sentiment_val_true_ = {}
df_sentiment_val_pred = {}

df_sentiment_test_true_ = {}
df_sentiment_test_pred = {}


# for aspect in lst_aspect:
for aspect in lst_aspect:
  pred_sentiment_train = sentiment_model[aspect].predict(padded_sentiment_train[aspect])
  pred_sentiment_val = sentiment_model[aspect].predict(padded_sentiment_val[aspect])
  pred_sentiment_test = sentiment_model[aspect].predict(padded_sentiment_test[aspect])

  df_sentiment_train_pred[aspect] = np.argmax(pred_sentiment_train, axis=1)
  df_sentiment_train_true_[aspect] = np.argmax(label_sentiment_train[aspect], axis = 1)

  df_sentiment_val_pred[aspect] = np.argmax(pred_sentiment_val, axis=1)
  df_sentiment_val_true_[aspect] = np.argmax(label_sentiment_val[aspect], axis = 1)

  df_sentiment_test_pred[aspect] = np.argmax(pred_sentiment_test, axis=1)
  df_sentiment_test_true_[aspect] = np.argmax(label_sentiment_test[aspect], axis=1)

sentiment_result_bigru_conv = {}
# for aspect in lst_aspect:
for aspect in lst_aspect:
    sentiment_result_bigru_conv[aspect] = classification_report(df_sentiment_test_true_[aspect].tolist(), df_sentiment_test_pred[aspect].tolist())
    print(sentiment_result_bigru_conv[aspect])

42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
              precision    recall  f1-score   support

           0       0.70      0.59      0.64        54
           1       0.64      0.41      0.50        22
           2       0.75      0.87      0.80        94

    accuracy                           0.72       170
   macro avg       0.69      0.62      0.65       170
weighted avg       0.72      0.72      0.71       170

              precision    recall  f1-score   support

           0       0.31      0.20      0.24        40
           1       0.74      0.75      0.74   

### BiLSTM+CONV

In [ ]:
def bilstm_conv_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val, epochs=70):

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_aspect_size,
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_aspect_matrix),
                  input_length=max_len,
                  trainable=True)(input)

  dropout1 = SpatialDropout1D(0.2)(embed)
  lstm = Bidirectional(LSTM(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

  avg_pool1 = GlobalAveragePooling1D()(conv)
  max_pool1 = GlobalMaxPooling1D()(conv)


  concat = Concatenate(axis=-1)([avg_pool1, max_pool1])

  # aspect
  aspect_dense2 = Dense(128, activation='relu')(concat)
  aspect_dropout1 = Dropout(0.2)(aspect_dense2)
  aspect_dense3 = Dense(64, activation='relu')(aspect_dropout1)
  aspect_dense4 = Dense(32, activation='relu')(aspect_dense3)
  aspect_dense5 = Dense(4, activation='sigmoid')(aspect_dense4)

  aspect_model = tf.keras.Model(inputs = input, outputs = aspect_dense5)
  aspect_model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model
  history = aspect_model.fit(x=padded_aspect_train,
    y=label_aspect_train.to_numpy(),

    validation_data=(padded_aspect_val,
      label_aspect_val.to_numpy()),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)

  return aspect_model

def bilstm_conv_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val, epochs=70):

  # polarity

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_sentiment_size[aspect],
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_sentiment_matrix[aspect]),
                  input_length=max_len,
                  trainable=True)(input)

  dropout1 = SpatialDropout1D(0.2)(embed)
  lstm = Bidirectional(LSTM(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

  avg_pool1 = GlobalAveragePooling1D()(conv)
  max_pool1 = GlobalMaxPooling1D()(conv)


  concat = Concatenate(axis=-1)([avg_pool1, max_pool1])

  sentiment_dense2 = Dense(128, activation='relu')(concat)
  sentiment_dropout1 = Dropout(0.2)(sentiment_dense2)
  sentiment_dense3 = Dense(64, activation='relu')(sentiment_dropout1)
  sentiment_dense4 = Dense(32, activation='relu')(sentiment_dense3)
  out_sentiment = Dense(units = 3, activation = 'softmax')(sentiment_dense4)

  sentiment_model[aspect] = tf.keras.Model(inputs = input, outputs = out_sentiment)
  sentiment_model[aspect].compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model

  history = sentiment_model[aspect].fit(x=padded_sentiment_train[aspect],
    y=label_sentiment_train[aspect],

    validation_data=(padded_sentiment_val[aspect],
      label_sentiment_val[aspect]),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)


  return sentiment_model[aspect]

In [ ]:
max_len=padded_aspect_train.shape[1]
aspect_model = bilstm_conv_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val)

save_path = '/kaggle/working/BiLSTM_CONV/stl_BiLSTM_CONV_aspect.h5'
aspect_model.save(save_path)

Epoch 1/70


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 106ms/step - acc: 0.1447 - loss: 0.6924 - val_acc: 0.2101 - val_loss: 0.6339
Epoch 2/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - acc: 0.2277 - loss: 0.6164 - val_acc: 0.3487 - val_loss: 0.5485
Epoch 3/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - acc: 0.3296 - loss: 0.5566 - val_acc: 0.4223 - val_loss: 0.5080
Epoch 4/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - acc: 0.4065 - loss: 0.5147 - val_acc: 0.4202 - val_loss: 0.4613
Epoch 5/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - acc: 0.4420 - loss: 0.4698 - val_acc: 0.5462 - val_loss: 0.4100
Epoch 6/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - acc: 0.5181 - loss: 0.4239 - val_acc: 0.6197 - val_loss: 0.3713
Epoch 7/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - acc: 0.5586 - loss: 0.3784 - val_acc: 0.5756 - val_loss: 0.3383
Epoch 8/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - acc: 0.5589 - loss: 0.3506 - val_acc: 0.5420 - val_loss: 0.3219
Epoch 9/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - acc: 0.5704 - loss: 0

In [ ]:
# load model
save_path = f'{path}/BiLSTM_CONV/stl_BiLSTM_CONV_aspect'
aspect_model = keras.models.load_model(save_path)

ValueError: File format not supported: filepath=/content/drive/MyDrive/ABSA/BiLSTM_CONV/stl_BiLSTM_CONV_aspect. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/drive/MyDrive/ABSA/BiLSTM_CONV/stl_BiLSTM_CONV_aspect, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
# predict
pred_train = aspect_model.predict(padded_aspect_train)
pred_val = aspect_model.predict(padded_aspect_val)
pred_test = aspect_model.predict(padded_aspect_test)

df_train_pred = round(pd.DataFrame(pred_train), 0)
df_train_true_ = label_aspect_train
df_train_pred.columns = df_train_true_.columns

df_val_pred = round(pd.DataFrame(pred_val), 0)
df_val_true_ = label_aspect_val
df_val_pred.columns = df_val_true_.columns

df_test_pred = round(pd.DataFrame(pred_test), 0)
df_test_true_ = label_aspect_test
df_test_pred.columns = df_test_true_.columns

for col in label_aspect_train.columns:
  print(col)
  aspect_result_bilstm_conv = classification_report(df_test_true_[col], df_test_pred[col])
  print(aspect_result_bilstm_conv)

119/119 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
is_character
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       306
           1       0.87      0.77      0.82       171

    accuracy                           0.88       477
   macro avg       0.88      0.85      0.86       477
weighted avg       0.88      0.88      0.88       477

is_content
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       273
           1       0.79      0.83      0.81       204

    accuracy                           0.83       477
   macro avg       0.83      0.83      0.83       477
weighted avg       0.84      0.83      0.83       477

is_scene
              precision    recall  f1-score   support

           0       0.84      0.86      0.85       309
           1       0.74      0.70      0.72       168

    accuracy               

- Stage 2 : Polarity

In [ ]:
lst_aspect = []
for col in label_aspect_train.columns.values:
    lst_aspect.append(col.split('_')[1])

In [ ]:
#tokenize stage 2
tokenized_sentiment_train = {}
tokenized_sentiment_val = {}
tokenized_sentiment_test = {}
tokenizer_sentiment = {}

for aspect in lst_aspect:
  tokenizer_sentiment[aspect], tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect] = get_tokenize(cmt_train[aspect], cmt_val[aspect], cmt_test[aspect])

#padding stage 2
padded_sentiment_train = {}
padded_sentiment_val = {}
padded_sentiment_test = {}
for aspect in lst_aspect:
  padded_sentiment_train[aspect], padded_sentiment_val[aspect], padded_sentiment_test[aspect] = get_padded(max_len, tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect])

#embedding matrix stage 2
vocab_sentiment_size = {}
embedding_sentiment_matrix = {}
for aspect in lst_aspect:
  vocab_sentiment_size[aspect]  = len(tokenizer_sentiment[aspect].word_index)+1
  embedding_sentiment_matrix[aspect] = get_embedding_matrix(embedding_dim, vocab_sentiment_size[aspect], tokenizer_sentiment[aspect])

In [ ]:
# training model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = bilstm_conv_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val)
  sentiment_model[aspect].save(f'/kaggle/working/BiLSTM_CONV/stl_BiLSTM_CONV_polarity_{aspect}.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 118ms/step - acc: 0.1668 - loss: 0.7242 - val_acc: 0.1548 - val_loss: 0.6752
Epoch 2/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - acc: 0.2327 - loss: 0.6750 - val_acc: 0.5417 - val_loss: 0.6472
Epoch 3/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - acc: 0.4393 - loss: 0.6513 - val_acc: 0.5893 - val_loss: 0.6226
Epoch 4/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.5383 - loss: 0.6299 - val_acc: 0.5714 - val_loss: 0.5928
Epoch 5/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.5261 - loss: 0.6082 - val_acc: 0.5714 - val_loss: 0.5600
Epoch 6/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.5519 - loss: 0.5831 - val_acc: 0.6845 - val_loss: 0.5269
Epoch 7/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.6244 - loss: 0.5470 - val_acc: 0.6964 - val_loss: 0.5049
Epoch 8/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.6535 - loss: 0.5238 - val_acc: 0.6905 - val_loss: 0.4921
Epoch 9/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - acc: 0.662

In [ ]:
# predict
df_sentiment_train_pred = {}
df_sentiment_train_true_ = {}

df_sentiment_val_true_ = {}
df_sentiment_val_pred = {}

df_sentiment_test_true_ = {}
df_sentiment_test_pred = {}

for aspect in lst_aspect:
  pred_sentiment_train = sentiment_model[aspect].predict(padded_sentiment_train[aspect])
  pred_sentiment_val = sentiment_model[aspect].predict(padded_sentiment_val[aspect])
  pred_sentiment_test = sentiment_model[aspect].predict(padded_sentiment_test[aspect])

  df_sentiment_train_pred[aspect] = np.argmax(pred_sentiment_train, axis=1)
  df_sentiment_train_true_[aspect] = np.argmax(label_sentiment_train[aspect], axis = 1)

  df_sentiment_val_pred[aspect] = np.argmax(pred_sentiment_val, axis=1)
  df_sentiment_val_true_[aspect] = np.argmax(label_sentiment_val[aspect], axis = 1)

  df_sentiment_test_pred[aspect] = np.argmax(pred_sentiment_test, axis=1)
  df_sentiment_test_true_[aspect] = np.argmax(label_sentiment_test[aspect], axis=1)

sentiment_result_bilstm_conv = {}
# for aspect in lst_aspect:
for aspect in lst_aspect:
    sentiment_result_bilstm_conv[aspect] = classification_report(df_sentiment_test_true_[aspect].tolist(), df_sentiment_test_pred[aspect].tolist())
    print(sentiment_result_bilstm_conv[aspect])

42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
              precision    recall  f1-score   support

           0       0.56      0.69      0.62        54
           1       0.64      0.32      0.42        22
           2       0.76      0.76      0.76        94

    accuracy                           0.68       170
   macro avg       0.65      0.59      0.60       170
weighted avg       0.68      0.68      0.67       170

              precision    recall  f1-score   support

           0       0.37      0.17      0.24        40
           1       0.74      0.70      0.72     

### BiLSTM+BiGRU+CONV

In [ ]:
def bilstm_bigru_conv_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val, epochs=70):

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_aspect_size,
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_aspect_matrix),
                  input_length=max_len,
                  trainable=True)(input)

  dropout1 = SpatialDropout1D(0.2)(embed)

  lstm = Bidirectional(LSTM(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv_lstm = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

  gru = Bidirectional(GRU(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv_gru = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(gru)


  avg_pool1 = GlobalAveragePooling1D()(conv_lstm)
  max_pool1 = GlobalMaxPooling1D()(conv_lstm)

  avg_pool2 = GlobalAveragePooling1D()(conv_gru)
  max_pool2 = GlobalMaxPooling1D()(conv_gru)


  concat = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])

  # aspect
  aspect_dense1 = Dense(units = 256, activation = 'relu')(concat)
  aspect_dense2 = Dense(128, activation='relu')(aspect_dense1)
  aspect_dropout1 = Dropout(0.2)(aspect_dense2)
  aspect_dense3 = Dense(64, activation='relu')(aspect_dropout1)
  aspect_dense4 = Dense(32, activation='relu')(aspect_dense3)
  aspect_dense5 = Dense(4, activation='sigmoid')(aspect_dense4)

  aspect_model = tf.keras.Model(inputs = input, outputs = aspect_dense5)
  aspect_model.compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model
  history = aspect_model.fit(x=padded_aspect_train,
    y=label_aspect_train.to_numpy(),

    validation_data=(padded_aspect_val,
      label_aspect_val.to_numpy()),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)

  return aspect_model

def bilstm_bigru_conv_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val, epochs=70):

  # polarity

  input = Input(shape=(max_len,))
  embed = Embedding(input_dim=vocab_sentiment_size[aspect],
                  output_dim=embedding_dim,
                  embeddings_initializer=Constant(embedding_sentiment_matrix[aspect]),
                  input_length=max_len,
                  trainable=True)(input)

  dropout1 = SpatialDropout1D(0.2)(embed)

  lstm = Bidirectional(LSTM(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv_lstm = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(lstm)

  gru = Bidirectional(GRU(units = 200, activation = 'tanh', return_sequences = True))(dropout1)
  conv_gru = Conv1D(128, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(gru)


  avg_pool1 = GlobalAveragePooling1D()(conv_lstm)
  max_pool1 = GlobalMaxPooling1D()(conv_lstm)

  avg_pool2 = GlobalAveragePooling1D()(conv_gru)
  max_pool2 = GlobalMaxPooling1D()(conv_gru)


  concat = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])

  sentiment_dense1 = Dense(units = 256, activation = 'relu')(concat)
  sentiment_dense2 = Dense(128, activation='relu')(sentiment_dense1)
  sentiment_dropout1 = Dropout(0.2)(sentiment_dense2)
  sentiment_dense3 = Dense(64, activation='relu')(sentiment_dropout1)
  sentiment_dense4 = Dense(32, activation='relu')(sentiment_dense3)
  out_sentiment = Dense(units = 3, activation = 'softmax')(sentiment_dense4)

  sentiment_model[aspect] = tf.keras.Model(inputs = input, outputs = out_sentiment)
  sentiment_model[aspect].compile(optimizer=Adam(learning_rate = 0.0001), loss='binary_crossentropy', metrics=['acc'])
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  # Fit model

  history = sentiment_model[aspect].fit(x=padded_sentiment_train[aspect],
    y=label_sentiment_train[aspect],

    validation_data=(padded_sentiment_val[aspect],
      label_sentiment_val[aspect]),
    batch_size = 128,
    epochs=epochs,
    callbacks = [callback],
    verbose=1)


  return sentiment_model[aspect]

In [ ]:
max_len=padded_aspect_train.shape[1]
aspect_model = bilstm_bigru_conv_aspect(vocab_aspect_size, embedding_dim, max_len, embedding_aspect_matrix, padded_aspect_train, padded_aspect_val, label_aspect_train, label_aspect_val)

save_path = f'/kaggle/working/BiLSTM_BiGRU_CONV/stl_BiLSTM_BiGRU_CONV_aspect.h5'
aspect_model.save(save_path)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 176ms/step - acc: 0.2651 - loss: 0.6839 - val_acc: 0.3803 - val_loss: 0.5940
Epoch 2/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - acc: 0.3979 - loss: 0.5782 - val_acc: 0.3403 - val_loss: 0.5027
Epoch 3/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 164ms/step - acc: 0.3771 - loss: 0.5103 - val_acc: 0.4286 - val_loss: 0.4383
Epoch 4/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - acc: 0.4234 - loss: 0.4482 - val_acc: 0.4769 - val_loss: 0.3852
Epoch 5/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - acc: 0.5096 - loss: 0.3929 - val_acc: 0.5147 - val_loss: 0.3422
Epoch 6/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 168ms/step - acc: 0.5428 - loss: 0.3499 - val_acc: 0.5252 - val_loss: 0.3237
Epoch 7/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - acc: 0.5773 - loss: 0.3265 - val_acc: 0.5966 - val_loss: 0.3042
Epoch 8/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - acc: 0.5792 - loss: 0.2965 - val_acc: 0.6113 - val_loss: 0.2953
Epoch 9/70
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - ac

In [ ]:
# load model
save_path = f'{path_model}/BiLSTM_BiGRU_CONV/stl_BiLSTM_BiGRU_CONV_aspect'
aspect_model = keras.models.load_model(save_path)

In [ ]:
# predict
pred_train = aspect_model.predict(padded_aspect_train)
pred_val = aspect_model.predict(padded_aspect_val)
pred_test = aspect_model.predict(padded_aspect_test)

df_train_pred = round(pd.DataFrame(pred_train), 0)
df_train_true_ = label_aspect_train
df_train_pred.columns = df_train_true_.columns

df_val_pred = round(pd.DataFrame(pred_val), 0)
df_val_true_ = label_aspect_val
df_val_pred.columns = df_val_true_.columns

df_test_pred = round(pd.DataFrame(pred_test), 0)
df_test_true_ = label_aspect_test
df_test_pred.columns = df_test_true_.columns

for col in label_aspect_train.columns:
  print(col)
  aspect_result_bilstm_bigru_conv = classification_report(df_test_true_[col], df_test_pred[col])
  print(aspect_result_bilstm_bigru_conv)

119/119 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
is_character
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       306
           1       0.85      0.78      0.82       171

    accuracy                           0.87       477
   macro avg       0.87      0.85      0.86       477
weighted avg       0.87      0.87      0.87       477

is_content
              precision    recall  f1-score   support

           0       0.84      0.85      0.84       273
           1       0.80      0.77      0.79       204

    accuracy                           0.82       477
   macro avg       0.82      0.81      0.82       477
weighted avg       0.82      0.82      0.82       477

is_scene
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       309
           1       0.78      0.69      0.73       168

    accuracy               

- Stage 2 : Polarity

In [ ]:
lst_aspect = []
for col in label_aspect_train.columns.values:
    lst_aspect.append(col.split('_')[1])

In [ ]:
#tokenize stage 2
tokenized_sentiment_train = {}
tokenized_sentiment_val = {}
tokenized_sentiment_test = {}
tokenizer_sentiment = {}

for aspect in lst_aspect:
  tokenizer_sentiment[aspect], tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect] = get_tokenize(cmt_train[aspect], cmt_val[aspect], cmt_test[aspect])

#padding stage 2
padded_sentiment_train = {}
padded_sentiment_val = {}
padded_sentiment_test = {}
for aspect in lst_aspect:
  padded_sentiment_train[aspect], padded_sentiment_val[aspect], padded_sentiment_test[aspect] = get_padded(max_len, tokenized_sentiment_train[aspect], tokenized_sentiment_val[aspect], tokenized_sentiment_test[aspect])

#embedding matrix stage 2
vocab_sentiment_size = {}
embedding_sentiment_matrix = {}
for aspect in lst_aspect:
  vocab_sentiment_size[aspect]  = len(tokenizer_sentiment[aspect].word_index)+1
  embedding_sentiment_matrix[aspect] = get_embedding_matrix(embedding_dim, vocab_sentiment_size[aspect], tokenizer_sentiment[aspect])

In [ ]:
path_model = "/kaggle/working"

In [ ]:
# training model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = bilstm_bigru_conv_polarity(aspect, vocab_sentiment_size, embedding_dim, max_len, embedding_sentiment_matrix, padded_sentiment_train, padded_sentiment_val, label_sentiment_train, label_sentiment_val)
  sentiment_model[aspect].save(f'/kaggle/working/BiLSTM_BiGRU_CONV/stl_BiLSTM_BiGRU_CONV_polarity_{aspect}.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 188ms/step - acc: 0.4553 - loss: 0.6260 - val_acc: 0.5714 - val_loss: 0.5579
Epoch 2/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - acc: 0.5261 - loss: 0.5850 - val_acc: 0.5714 - val_loss: 0.5455
Epoch 3/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 144ms/step - acc: 0.5360 - loss: 0.5587 - val_acc: 0.6488 - val_loss: 0.5231
Epoch 4/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - acc: 0.5949 - loss: 0.5478 - val_acc: 0.6905 - val_loss: 0.5019
Epoch 5/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 146ms/step - acc: 0.6671 - loss: 0.5073 - val_acc: 0.6845 - val_loss: 0.4808
Epoch 6/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - acc: 0.6664 - loss: 0.4780 - val_acc: 0.6905 - val_loss: 0.4671
Epoch 7/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - acc: 0.6634 - loss: 0.4538 - val_acc: 0.6905 - val_loss: 0.4545
Epoch 8/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - acc: 0.6902 - loss: 0.4256 - val_acc: 0.7024 - val_loss: 0.4334
Epoch 9/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 148ms/step - ac

In [ ]:
# load model
sentiment_model = {}
for aspect in lst_aspect:
  sentiment_model[aspect] = keras.models.load_model(f'{path_model}/BiLSTM_BiGRU_CONV/stl_BiLSTM_BiGRU_CONV_polarity_{aspect}.h5')

In [ ]:
# predict
df_sentiment_train_pred = {}
df_sentiment_train_true_ = {}

df_sentiment_val_true_ = {}
df_sentiment_val_pred = {}

df_sentiment_test_true_ = {}
df_sentiment_test_pred = {}


# for aspect in lst_aspect:
for aspect in lst_aspect:
  pred_sentiment_train = sentiment_model[aspect].predict(padded_sentiment_train[aspect])
  pred_sentiment_val = sentiment_model[aspect].predict(padded_sentiment_val[aspect])
  pred_sentiment_test = sentiment_model[aspect].predict(padded_sentiment_test[aspect])

  df_sentiment_train_pred[aspect] = np.argmax(pred_sentiment_train, axis=1)
  df_sentiment_train_true_[aspect] = np.argmax(label_sentiment_train[aspect], axis = 1)

  df_sentiment_val_pred[aspect] = np.argmax(pred_sentiment_val, axis=1)
  df_sentiment_val_true_[aspect] = np.argmax(label_sentiment_val[aspect], axis = 1)

  df_sentiment_test_pred[aspect] = np.argmax(pred_sentiment_test, axis=1)
  df_sentiment_test_true_[aspect] = np.argmax(label_sentiment_test[aspect], axis=1)

sentiment_result_bilstm_bigru_conv = {}
# for aspect in lst_aspect:
for aspect in lst_aspect:
    sentiment_result_bilstm_bigru_conv[aspect] = classification_report(df_sentiment_test_true_[aspect].tolist(), df_sentiment_test_pred[aspect].tolist())
    print(sentiment_result_bilstm_bigru_conv[aspect])

42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
              precision    recall  f1-score   support

           0       0.76      0.63      0.69        54
           1       0.45      0.41      0.43        22
           2       0.73      0.82      0.77        94

    accuracy                           0.71       170
   macro avg       0.65      0.62      0.63       170
weighted avg       0.70      0.71      0.70       170

              precision    recall  f1-score   support

           0       0.25      0.35      0.29        40
           1       0.76      0.47      0.58   